# Import Data

In [1]:
import requests
#import bs4
import time
from selenium import webdriver

import pandas as pd
import numpy as np

# Web Scraping with Selenium

In [2]:
driver = webdriver.Chrome('/Users/hyeonjeonglee/Downloads/chromedriver')  # Optional argument, if not specified will search path.
driver.get('https://www.worldometers.info/coronavirus/');
time.sleep(5) # Let the user actually see something!

# Get data from yesterday
driver.find_element_by_xpath("//a[@href='#nav-yesterday']").click()


# Grab Columns 
column_name = []
for item in driver.find_elements_by_tag_name('th'):
    if "display: none" not in item.get_attribute("style"):
        column = item.text
        column_name.append(column)

column_name = [x for x in column_name if x not in ['']]
column_name = [x.replace('\n',' ') for x in column_name]
    
print(column_name)

# Grab Rows #

## 1. Grab all data from each row


table_id = driver.find_element_by_id('main_table_countries_yesterday')
tbody = table_id.find_element_by_tag_name('tbody')
#print(tbody.get_property('attributes'))


tr_list = []
for item in tbody.find_elements_by_tag_name('tr'):
    if "display: none" not in item.get_attribute('style'):
        tr_list.append(item)

tr_list_clean = tr_list[1:217]

# print(tr_list_clean[0].text)
# print(tr_list_clean[-1].text)

# print(len(tr_list_clean))


## 2. Create a list of list where each list contains data for each country

list_of_list = []

def row_data(tr_value):
    row_items=[]
    #append row data to row_items  
    for item in tr_value.find_elements_by_tag_name('td'):
        if len(row_items) <=13:
            row_items.append(item.text)
            
    return row_items

for item in tr_list_clean:
    row = row_data(item)
    list_of_list.append(row)

#print(list_of_list)


time.sleep(5) # Let the user actually see something!

driver.quit()

['#', 'Country, Other', 'Total Cases', 'New Cases', 'Total Deaths', 'New Deaths', 'Total Recovered', 'Active Cases', 'Serious, Critical', 'Tot Cases/ 1M pop', 'Deaths/ 1M pop', 'Total Tests', 'Tests/ 1M pop', 'Population']


# Create dataframe called covid_all

add each item as rows in dataframe

In [3]:
covid_all = pd.DataFrame(list_of_list,columns=column_name)    

In [4]:
covid_all.set_index('#', inplace=True)

replace 'NA' strings to -1 to convert to integer

In [5]:
for i in covid_all.iloc[:,1:]:
    covid_all[i] = covid_all[i].replace('N/A','-1')
    covid_all[i] = covid_all[i].replace('','-1')
    covid_all[i] = covid_all[i].replace(' ','-1')

In [6]:
covid_all.tail()

,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/ 1M pop,Deaths/ 1M pop,Total Tests,Tests/ 1M pop,Population
#,,,,,,,,,,,,,
211,Caribbean Netherlands,7,-1,-1,-1,7,0,-1,267,-1,424,"16,176","26,212"
212,St. Barth,6,-1,-1,-1,6,0,-1,608,-1,152,"15,391","9,876"
213,Lesotho,4,-1,-1,-1,2,2,-1,2,-1,"1,515",707,"2,141,487"
214,Anguilla,3,-1,-1,-1,3,0,-1,200,-1,93,"6,201","14,997"
215,Saint Pierre Miquelon,1,-1,-1,-1,1,0,-1,173,-1,-1,-1,"5,795"


In [7]:
covid_all.head()

,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/ 1M pop,Deaths/ 1M pop,Total Tests,Tests/ 1M pop,Population
#,,,,,,,,,,,,,
1,USA,"2,162,228","+20,004","117,858",+331,"870,050","1,174,320","16,704","6,534",356,"24,794,617","74,927","330,917,584"
2,Brazil,"867,882","+17,086","43,389",+598,"437,512","386,981","8,318","4,084",204,"1,499,041","7,055","212,492,226"
3,Russia,"528,964","+8,835","6,948",+119,"280,050","241,966","2,300","3,625",48,"14,880,172","101,967","145,931,893"
4,India,"332,783","+11,157","9,520",+321,"169,689","153,574","8,944",241,7,"5,658,614","4,102","1,379,381,861"
5,UK,"295,889","+1,514","41,698",+36,-1,-1,392,"4,360",614,"6,772,602","99,787","67,870,494"


Remove commas and + from numbers

In [8]:
for i in covid_all.iloc[:,1:]:
    covid_all[i] = covid_all[i].str.replace(",","")
    #print(covid_all[i].tail(5))
    covid_all[i] = covid_all[i].str.replace("+","").astype(float)
    # print(i)
  

In [9]:
covid_all.tail()

,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/ 1M pop,Deaths/ 1M pop,Total Tests,Tests/ 1M pop,Population
#,,,,,,,,,,,,,
211,Caribbean Netherlands,7.0,-1.0,-1.0,-1.0,7.0,0.0,-1.0,267.0,-1.0,424.0,16176.0,26212.0
212,St. Barth,6.0,-1.0,-1.0,-1.0,6.0,0.0,-1.0,608.0,-1.0,152.0,15391.0,9876.0
213,Lesotho,4.0,-1.0,-1.0,-1.0,2.0,2.0,-1.0,2.0,-1.0,1515.0,707.0,2141487.0
214,Anguilla,3.0,-1.0,-1.0,-1.0,3.0,0.0,-1.0,200.0,-1.0,93.0,6201.0,14997.0
215,Saint Pierre Miquelon,1.0,-1.0,-1.0,-1.0,1.0,0.0,-1.0,173.0,-1.0,-1.0,-1.0,5795.0


In [10]:
covid_all.head()

,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/ 1M pop,Deaths/ 1M pop,Total Tests,Tests/ 1M pop,Population
#,,,,,,,,,,,,,
1,USA,2162228.0,20004.0,117858.0,331.0,870050.0,1174320.0,16704.0,6534.0,356.0,24794617.0,74927.0,3.309176e+08
2,Brazil,867882.0,17086.0,43389.0,598.0,437512.0,386981.0,8318.0,4084.0,204.0,1499041.0,7055.0,2.124922e+08
3,Russia,528964.0,8835.0,6948.0,119.0,280050.0,241966.0,2300.0,3625.0,48.0,14880172.0,101967.0,1.459319e+08
4,India,332783.0,11157.0,9520.0,321.0,169689.0,153574.0,8944.0,241.0,7.0,5658614.0,4102.0,1.379382e+09
5,UK,295889.0,1514.0,41698.0,36.0,-1.0,-1.0,392.0,4360.0,614.0,6772602.0,99787.0,6.787049e+07


change -1 to null values

In [11]:
covid_all.replace(-1,np.nan,inplace=True)

In [12]:
covid_all.head()

,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/ 1M pop,Deaths/ 1M pop,Total Tests,Tests/ 1M pop,Population
#,,,,,,,,,,,,,
1,USA,2162228.0,20004.0,117858.0,331.0,870050.0,1174320.0,16704.0,6534.0,356.0,24794617.0,74927.0,3.309176e+08
2,Brazil,867882.0,17086.0,43389.0,598.0,437512.0,386981.0,8318.0,4084.0,204.0,1499041.0,7055.0,2.124922e+08
3,Russia,528964.0,8835.0,6948.0,119.0,280050.0,241966.0,2300.0,3625.0,48.0,14880172.0,101967.0,1.459319e+08
4,India,332783.0,11157.0,9520.0,321.0,169689.0,153574.0,8944.0,241.0,7.0,5658614.0,4102.0,1.379382e+09
5,UK,295889.0,1514.0,41698.0,36.0,NaN,NaN,392.0,4360.0,614.0,6772602.0,99787.0,6.787049e+07
